In [31]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [32]:
print(tf.__version__)

2.4.1


In [33]:
dataset_path = keras.utils.get_file("auto-mpg.data", "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
dataset_path

'/root/.keras/datasets/auto-mpg.data'

In [34]:
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight','Acceleration', 'Model Year', 'Origin']
raw_dataset = pd.read_csv(dataset_path, names=column_names, na_values = "?", comment='\t', sep=" ", skipinitialspace=True)

In [35]:
dataset = raw_dataset.copy()
dataset.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,Origin
393,27.0,4,140.0,86.0,2790.0,15.6,82,1
394,44.0,4,97.0,52.0,2130.0,24.6,82,2
395,32.0,4,135.0,84.0,2295.0,11.6,82,1
396,28.0,4,120.0,79.0,2625.0,18.6,82,1
397,31.0,4,119.0,82.0,2720.0,19.4,82,1


In [36]:
dataset.isna().sum()
dataset = dataset.dropna()

In [37]:
origin = dataset.pop('Origin')
dataset['USA'] = (origin == 1)*1.0
dataset['Europe'] = (origin == 2)*1.0
dataset['Japan'] = (origin == 3)*1.0
dataset.tail()

,MPG,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,USA,Europe,Japan
393,27.0,4,140.0,86.0,2790.0,15.6,82,1.0,0.0,0.0
394,44.0,4,97.0,52.0,2130.0,24.6,82,0.0,1.0,0.0
395,32.0,4,135.0,84.0,2295.0,11.6,82,1.0,0.0,0.0
396,28.0,4,120.0,79.0,2625.0,18.6,82,1.0,0.0,0.0
397,31.0,4,119.0,82.0,2720.0,19.4,82,1.0,0.0,0.0


In [38]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [44]:
train_stats = train_dataset.describe()
train_stats

,Cylinders,Displacement,Horsepower,Weight,Acceleration,Model Year,USA,Europe,Japan
count,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000,314.000000
mean,5.477707,195.318471,104.869427,2990.251592,15.559236,75.898089,0.624204,0.178344,0.197452
std,1.699788,104.331589,38.096214,843.898596,2.789230,3.675642,0.485101,0.383413,0.398712
min,3.000000,68.000000,46.000000,1649.000000,8.000000,70.000000,0.000000,0.000000,0.000000
25%,4.000000,105.500000,76.250000,2256.500000,13.800000,73.000000,0.000000,0.000000,0.000000
50%,4.000000,151.000000,94.500000,2822.500000,15.500000,76.000000,1.000000,0.000000,0.000000
75%,8.000000,265.750000,128.000000,3608.000000,17.200000,79.000000,1.000000,0.000000,0.000000
max,8.000000,455.000000,225.000000,5140.000000,24.800000,82.000000,1.000000,1.000000,1.000000


In [48]:
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

KeyError: ignored

In [46]:
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
Cylinders,314.0,5.477707,1.699788,3.0,4.00,4.0,8.00,8.0
Displacement,314.0,195.318471,104.331589,68.0,105.50,151.0,265.75,455.0
Horsepower,314.0,104.869427,38.096214,46.0,76.25,94.5,128.00,225.0
Weight,314.0,2990.251592,843.898596,1649.0,2256.50,2822.5,3608.00,5140.0
Acceleration,314.0,15.559236,2.789230,8.0,13.80,15.5,17.20,24.8
Model Year,314.0,75.898089,3.675642,70.0,73.00,76.0,79.00,82.0
USA,314.0,0.624204,0.485101,0.0,0.00,1.0,1.00,1.0
Europe,314.0,0.178344,0.383413,0.0,0.00,0.0,0.00,1.0
Japan,314.0,0.197452,0.398712,0.0,0.00,0.0,0.00,1.0


# Normalización de valores

(x - mean of elements in the column)/ standard deviation

In [47]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
  
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

# Construir Modelo

In [49]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [50]:
model = build_model()

In [54]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 64)                640       
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 4,865
Trainable params: 4,865
Non-trainable params: 0
_________________________________________________________________


In [55]:
EPOCHS = 1000

In [56]:
history = model.fit(
    normed_train_data, train_labels, epochs=EPOCHS
)

Epoch 1/1000
10/10 [==============================] - 1s 2ms/step - loss: 566.5935 - mae: 22.5109 - mse: 566.5935
Epoch 2/1000
10/10 [==============================] - 0s 2ms/step - loss: 510.4108 - mae: 21.4600 - mse: 510.4108
Epoch 3/1000
10/10 [==============================] - 0s 2ms/step - loss: 426.4228 - mae: 19.5709 - mse: 426.4228
Epoch 4/1000
10/10 [==============================] - 0s 2ms/step - loss: 350.0529 - mae: 17.6797 - mse: 350.0529
Epoch 5/1000
10/10 [==============================] - 0s 2ms/step - loss: 287.9802 - mae: 15.7711 - mse: 287.9802
Epoch 6/1000
10/10 [==============================] - 0s 2ms/step - loss: 207.8502 - mae: 13.1019 - mse: 207.8502
Epoch 7/1000
10/10 [==============================] - 0s 2ms/step - loss: 142.1403 - mae: 10.2833 - mse: 142.1403
Epoch 8/1000
10/10 [==============================] - 0s 2ms/step - loss: 86.3772 - mae: 7.5571 - mse: 86.3772
Epoch 9/1000
10/10 [==============================] - 0s 2ms/step - loss: 65.2003 - mae: 6.

In [57]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=0)
print(loss)
print(mae)
print(mse)

6.550350666046143
1.8969390392303467
6.550351142883301


In [58]:
test_predictions = model.predict(normed_test_data).flatten()
test_predictions

array([13.428785, 10.672426,  9.565816, 24.964603, 18.514984, 13.585358,
       13.86188 , 12.516123, 19.469334, 30.974241, 24.343613, 18.507153,
       16.201212, 25.679527, 12.86051 , 15.26123 , 14.238501, 14.083654,
       18.095078, 13.467895, 11.59029 , 25.85625 , 20.065369, 21.698647,
       30.876976, 24.015743, 16.73579 , 24.146847, 18.173923, 15.757139,
       29.383392, 25.427425, 19.436235, 21.466125, 26.830488, 15.681059,
       19.897968, 28.47852 , 30.225609, 19.23211 , 31.248558, 29.5518  ,
       14.947652, 40.685863, 37.484787, 38.523876, 20.522947, 20.242474,
       20.812319, 26.420616, 33.00162 , 17.342775, 32.061287, 18.755518,
       17.423903, 19.014301, 35.01435 , 36.787876, 21.421932, 26.134512,
       44.684147, 33.02838 , 27.268913, 26.274937, 33.499413, 39.22811 ,
       36.744877, 36.22745 , 39.128994, 34.04813 , 19.411215, 32.627197,
       30.065945, 30.767296, 41.28027 , 35.607754, 38.29678 , 31.165407],
      dtype=float32)

# Guardar Modelo

In [61]:
keras_file = 'automobile.h5'
tf.keras.models.save_model(model, keras_file)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tfmodel = converter.convert()
open('automobile.tflite', 'wb').write(tfmodel)

INFO:tensorflow:Assets written to: /tmp/tmpos30c8v0/assets


INFO:tensorflow:Assets written to: /tmp/tmpos30c8v0/assets


21068